# Demucs Custom Training

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import time

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

In [2]:
import musdb
import random

In [3]:
base_dir = os.path.join(os.getcwd(), "venvs/demucs")
data_base_dir = os.path.join(base_dir, "musdb18")

In [ ]:
BATCH_SIZE = 16

In [4]:
mus_train= musdb.DB(root=data_base_dir, subsets='train')
mus_test = musdb.DB(root=data_base_dir, subsets='test')
print('Number of files in train sets:', len(mus_train))
print('Number of files in test sets:', len(mus_test))

Number of files in train sets: 100
Number of files in test sets: 50


In [9]:
def gen_ds(file, length=44100*11, long=44100*10, strides=44100, random_time=random.randint(0, 44100)):
    input_mixture = file.stems[0]
    target_mixtures = np.column_stack(file.stems[1:])
    
    ds_inp = tf.data.Dataset.from_tensor_slices((input_mixture))
    ds_inp = ds_inp.window(length, shift=strides, drop_remainder=True)
    ds_inp = ds_inp.flat_map(lambda windows: windows.batch(length))
    ds_inp = ds_inp.map(lambda windows: (windows[random_time:(random_time+long), :]))

    ds_tar = tf.data.Dataset.from_tensor_slices((target_mixtures))
    ds_tar = ds_tar.window(length, shift=strides, drop_remainder=True)
    ds_tar = ds_tar.flat_map(lambda windows: windows.batch(length))
    ds_tar = ds_tar.map(lambda windows: (windows[random_time:(random_time+long), :]))

    ds_total = [ds_inp, ds_tar]
    total_ds = tf.data.Dataset.zip(tuple(ds_total))
    total_ds = total_ds.batch(BATCH_SIZE)
    
    return total_ds

In [10]:
# %%time
# for i, file in enumerate(mus_train):
#     gen_ds(file)
#     print('.', end='')
#     if i == 3:
#         break

....Wall time: 13.2 s
